# NexaAI NPU Inference Examples

This notebook demonstrates how to use the NexaAI SDK for various AI inference tasks on NPU devices, including:

- **LLM (Large Language Model)**: Text generation and conversation
- **VLM (Vision Language Model)**: Multimodal understanding and generation
- **Embedder**: Text vectorization and similarity computation
- **Reranker**: Document reranking

## Prerequisites

- Python 3.10+
- Windows ARM64 (Snapdragon X Elite) or NPU-capable device
- NexaAI SDK installed: `pip install nexaai`

## Model Download

Before running the examples, download the corresponding NPU-optimized models:

```bash
# LLM model
nexa pull NexaAI/Llama3.2-3B-NPU-Turbo

# VLM model  
nexa pull NexaAI/OmniNeural-4B

# Embedder model
nexa pull NexaAI/embeddinggemma-300m-npu

# Reranker model
nexa pull NexaAI/jina-v2-rerank-npu
```


In [ ]:
# Import necessary libraries
import os
import io
import numpy as np
from typing import List, Optional

# NexaAI SDK imports
from nexaai.llm import LLM, GenerationConfig
from nexaai.vlm import VLM
from nexaai.embedder import Embedder, EmbeddingConfig
from nexaai.rerank import Reranker, RerankConfig
from nexaai.common import ModelConfig, ChatMessage, MultiModalMessage, MultiModalMessageContent

print("NexaAI SDK imported successfully!")
print("Ready to start NPU inference examples...")


## 1. LLM (Large Language Model) NPU Inference

Using NPU-accelerated large language models for text generation and conversation. Llama3.2-3B-NPU-Turbo is specifically optimized for NPU.


In [ ]:
# LLM NPU Inference Example
def llm_npu_example():
    """LLM NPU inference example"""
    print("=== LLM NPU Inference Example ===")
    
    # Model configuration
    model_name = "NexaAI/Llama3.2-3B-NPU-Turbo"
    plugin_id = "npu"
    device = "npu"
    max_tokens = 100
    system_message = "You are a helpful assistant."
    
    print(f"Loading model: {model_name}")
    print(f"Using plugin: {plugin_id}")
    print(f"Device: {device}")
    
    # Create model instance
    m_cfg = ModelConfig()
    llm = LLM.from_(model_name, plugin_id=plugin_id, device_id=device, m_cfg=m_cfg)
    
    # Create conversation history
    conversation = [ChatMessage(role="system", content=system_message)]
    
    # Example conversations
    test_prompts = [
        "What is artificial intelligence?",
        "Explain the benefits of on-device AI processing.",
        "How does NPU acceleration work?"
    ]
    
    for i, prompt in enumerate(test_prompts, 1):
        print(f"\n--- Conversation {i} ---")
        print(f"User: {prompt}")
        
        # Add user message
        conversation.append(ChatMessage(role="user", content=prompt))
        
        # Apply chat template
        formatted_prompt = llm.apply_chat_template(conversation)
        
        # Generate response
        print("Assistant: ", end="", flush=True)
        response_buffer = io.StringIO()
        
        for token in llm.generate_stream(formatted_prompt, g_cfg=GenerationConfig(max_tokens=max_tokens)):
            print(token, end="", flush=True)
            response_buffer.write(token)
        
        # Get profiling data
        profiling_data = llm.get_profiling_data()
        if profiling_data:
            print(f"\nProfiling data: {profiling_data}")
        
        # Add assistant response to conversation history
        conversation.append(ChatMessage(role="assistant", content=response_buffer.getvalue()))
        print("\n" + "="*50)

# Run LLM example
llm_npu_example()


## 2. VLM (Vision Language Model) NPU Inference

Using NPU-accelerated vision language models for multimodal understanding and generation. OmniNeural-4B supports joint processing of images and text.


In [ ]:
# VLM NPU Inference Example
def vlm_npu_example():
    """VLM NPU inference example"""
    print("=== VLM NPU Inference Example ===")
    
    # Model configuration
    model_name = "NexaAI/OmniNeural-4B"
    plugin_id = "npu"
    device = "npu"
    max_tokens = 100
    system_message = "You are a helpful assistant that can understand images and text."
    
    print(f"Loading model: {model_name}")
    print(f"Using plugin: {plugin_id}")
    print(f"Device: {device}")
    
    # Create model instance
    m_cfg = ModelConfig()
    vlm = VLM.from_(name_or_path=model_name, m_cfg=m_cfg, plugin_id=plugin_id, device_id=device)
    
    # Create conversation history
    conversation = [MultiModalMessage(role="system", 
                                    content=[MultiModalMessageContent(type="text", text=system_message)])]
    
    # Example multimodal conversations
    test_cases = [
        {
            "text": "What do you see in this image?",
            "image_path": None  # Replace with actual image path if available
        },
        {
            "text": "Describe the main objects and their relationships.",
            "image_path": None
        },
        {
            "text": "What is artificial intelligence and how does it relate to computer vision?",
            "image_path": None
        }
    ]
    
    for i, case in enumerate(test_cases, 1):
        print(f"\n--- Multimodal Conversation {i} ---")
        print(f"User: {case['text']}")
        
        # Build message content
        contents = [MultiModalMessageContent(type="text", text=case['text'])]
        
        # Add image content if available
        if case['image_path'] and os.path.exists(case['image_path']):
            contents.append(MultiModalMessageContent(type="image", text=case['image_path']))
            print(f"Including image: {case['image_path']}")
        
        # Add user message
        conversation.append(MultiModalMessage(role="user", content=contents))
        
        # Apply chat template
        formatted_prompt = vlm.apply_chat_template(conversation)
        
        # Generate response
        print("Assistant: ", end="", flush=True)
        response_buffer = io.StringIO()
        
        # Prepare image and audio paths
        image_paths = [case['image_path']] if case['image_path'] and os.path.exists(case['image_path']) else None
        audio_paths = None
        
        for token in vlm.generate_stream(formatted_prompt, 
                                       g_cfg=GenerationConfig(max_tokens=max_tokens, 
                                                            image_paths=image_paths, 
                                                            audio_paths=audio_paths)):
            print(token, end="", flush=True)
            response_buffer.write(token)
        
        # Get profiling data
        profiling_data = vlm.get_profiling_data()
        if profiling_data:
            print(f"\nProfiling data: {profiling_data}")
        
        # Add assistant response to conversation history
        conversation.append(MultiModalMessage(role="assistant", 
                                            content=[MultiModalMessageContent(type="text", text=response_buffer.getvalue())]))
        print("\n" + "="*50)

# Run VLM example
vlm_npu_example()


## 3. Embedder NPU Inference

Using NPU-accelerated embedding models for text vectorization and similarity computation. embeddinggemma-300m-npu is a lightweight embedding model specifically optimized for NPU.


In [ ]:
# Embedder NPU Inference Example
def embedder_npu_example():
    """Embedder NPU inference example"""
    print("=== Embedder NPU Inference Example ===")
    
    # Model configuration
    model_name = "NexaAI/embeddinggemma-300m-npu"
    plugin_id = "npu"
    batch_size = 2
    
    print(f"Loading model: {model_name}")
    print(f"Using plugin: {plugin_id}")
    print(f"Batch size: {batch_size}")
    
    # Create embedder instance
    embedder = Embedder.from_(name_or_path=model_name, plugin_id=plugin_id)
    print('Embedder loaded successfully!')
    
    # Get embedding dimension
    dim = embedder.get_embedding_dim()
    print(f"Embedding dimension: {dim}")
    
    # Example texts
    texts = [
        "On-device AI is a type of AI that is processed on the device itself, rather than in the cloud.",
        "Nexa AI allows you to run state-of-the-art AI models locally on CPU, GPU, or NPU.",
        "A ragdoll is a breed of cat that is known for its long, flowing hair and gentle personality.",
        "The capital of France is Paris.",
        "NPU acceleration provides significant performance improvements for AI workloads."
    ]
    
    query = "what is on device AI"
    
    print(f"\n=== Generating Embeddings ===")
    print(f"Processing {len(texts)} texts...")
    
    # Generate embeddings
    embeddings = embedder.generate(
        texts=texts, 
        config=EmbeddingConfig(batch_size=batch_size)
    )
    
    print(f"Successfully generated {len(embeddings)} embeddings")
    
    # Display embedding information
    print(f"\n=== Embedding Details ===")
    for i, (text, embedding) in enumerate(zip(texts, embeddings)):
        print(f"\nText {i+1}:")
        print(f"  Content: {text}")
        print(f"  Embedding dimension: {len(embedding)}")
        print(f"  First 10 elements: {embedding[:10]}")
        print("-" * 70)
    
    # Query processing
    print(f"\n=== Query Processing ===")
    print(f"Query: '{query}'")
    
    query_embedding = embedder.generate(
        texts=[query], 
        config=EmbeddingConfig(batch_size=1)
    )[0]
    
    print(f"Query embedding dimension: {len(query_embedding)}")
    
    # Similarity analysis
    print(f"\n=== Similarity Analysis (Inner Product) ===")
    similarities = []
    
    for i, (text, embedding) in enumerate(zip(texts, embeddings)):
        query_vec = np.array(query_embedding)
        text_vec = np.array(embedding)
        inner_product = np.dot(query_vec, text_vec)
        similarities.append((i, text, inner_product))
        
        print(f"\nText {i+1}:")
        print(f"  Content: {text}")
        print(f"  Inner product with query: {inner_product:.6f}")
        print("-" * 70)
    
    # Sort and display most similar texts
    similarities.sort(key=lambda x: x[2], reverse=True)
    
    print(f"\n=== Similarity Ranking Results ===")
    for rank, (idx, text, score) in enumerate(similarities, 1):
        print(f"Rank {rank}: [{score:.6f}] {text}")
    
    return embeddings, query_embedding, similarities

# Run Embedder example
embeddings, query_emb, similarities = embedder_npu_example()


## 4. Reranker NPU Inference

Using NPU-accelerated reranking models for document reranking. jina-v2-rerank-npu can perform precise similarity-based document ranking based on queries.


In [ ]:
# Reranker NPU Inference Example
def reranker_npu_example():
    """Reranker NPU inference example"""
    print("=== Reranker NPU Inference Example ===")
    
    # Model configuration
    model_name = "NexaAI/jina-v2-rerank-npu"
    plugin_id = "npu"
    batch_size = 4
    
    print(f"Loading model: {model_name}")
    print(f"Using plugin: {plugin_id}")
    print(f"Batch size: {batch_size}")
    
    # Create reranker instance
    reranker = Reranker.from_(name_or_path=model_name, plugin_id=plugin_id)
    print('Reranker loaded successfully!')
    
    # Example queries and documents
    queries = [
        "Where is on-device AI?",
        "What is NPU acceleration?",
        "How does machine learning work?",
        "Tell me about computer vision"
    ]
    
    documents = [
        "On-device AI is a type of AI that is processed on the device itself, rather than in the cloud.",
        "NPU acceleration provides significant performance improvements for AI workloads on specialized hardware.",
        "Edge computing brings computation and data storage closer to the sources of data.",
        "A ragdoll is a breed of cat that is known for its long, flowing hair and gentle personality.",
        "The capital of France is Paris, a beautiful city known for its art and culture.",
        "Machine learning is a subset of artificial intelligence that enables computers to learn without being explicitly programmed.",
        "Computer vision is a field of artificial intelligence that trains computers to interpret and understand visual information.",
        "Deep learning uses neural networks with multiple layers to model and understand complex patterns in data."
    ]
    
    print(f"\n=== Document Reranking Test ===")
    print(f"Number of documents: {len(documents)}")
    
    # Rerank for each query
    for i, query in enumerate(queries, 1):
        print(f"\n--- Query {i} ---")
        print(f"Query: '{query}'")
        print("-" * 50)
        
        # Perform reranking
        scores = reranker.rerank(
            query=query, 
            documents=documents, 
            config=RerankConfig(batch_size=batch_size)
        )
        
        # Create (document, score) pairs and sort
        doc_scores = list(zip(documents, scores))
        doc_scores.sort(key=lambda x: x[1], reverse=True)
        
        # Display ranking results
        print("Reranking results:")
        for rank, (doc, score) in enumerate(doc_scores, 1):
            print(f"  {rank:2d}. [{score:.4f}] {doc}")
        
        # Display most relevant documents
        print(f"\nMost relevant documents (top 3):")
        for rank, (doc, score) in enumerate(doc_scores[:3], 1):
            print(f"  {rank}. {doc}")
        
        print("=" * 80)
    
    return reranker

# Run Reranker example
reranker = reranker_npu_example()
